<a href="https://colab.research.google.com/github/jaesuun/sjtg_project/blob/main/mission1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from PIL import Image
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
import torchvision
from torchvision import transforms

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [20]:
%cd /content/drive/MyDrive/데이터셋
!unzip -qq "/content/drive/MyDrive/데이터셋/kfood_val.zip"

/content/drive/MyDrive/데이터셋


In [25]:
import os

# 압축 파일이 해제된 폴더 경로
extracted_folder = "/content/drive/MyDrive/데이터셋/val"

# 해당 폴더 내의 파일 수 확인
file_count = len(os.listdir(extracted_folder))

print("압축 해제된 폴더 내의 파일 수:", file_count)


압축 해제된 폴더 내의 파일 수: 42
